In [1]:
import cv2
import os
import shutil
import numpy as np
import zipfile
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Dense, Flatten, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from tqdm import tqdm
from PIL import Image, ImageFilter, ImageEnhance
from keras.optimizers import Adam
from keras.regularizers import l2

In [2]:
# Unzipping the image dataset
zip_ref = zipfile.ZipFile('original.zip', 'r')
zip_ref.extractall('dataset')  # Extracting to the 'dataset' folder
zip_ref.close()

In [3]:
# Set the rotation parameters
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=90,
    fill_mode='nearest'
)

# Create the image data generator
if not os.path.exists('dataset/rotated'):
    os.makedirs('dataset/rotated')

# Specify the path to the original image
original_dir = 'dataset/original'

# Specify the number of images to generate
total_images = 20

In [ ]:
original_dir = 'dataset/original'
save_dir = 'dataset/rotated'

# Ensure the save directory exists
os.makedirs(save_dir, exist_ok=True)

# Number of rotated images to generate per original image
num_rotations = 10

# Process each image in the original directory
for filename in os.listdir(original_dir):
    if filename.endswith('.jpg'):
        # Load the image from file
        img_path = os.path.join(original_dir, filename)
        img = load_img(img_path)
        x = img_to_array(img)
        x = x.reshape((1,) + x.shape)  # Reshape to (1, height, width, channels)

        # Create and save rotated images
        i = 0
        for batch in datagen.flow(x, batch_size=1, save_to_dir=save_dir, save_prefix='rot', save_format='jpeg'):
            i += 1
            if i >= num_rotations:
                break

In [6]:
# Define data paths
original_dir = 'dataset/original'
rotated_dir = 'dataset/rotated'
train_dir = 'dataset/train'
validation_dir = 'dataset/validation'

# Create directories for training and validation
for category in ['original', 'rotated']:
    os.makedirs(os.path.join(train_dir, category), exist_ok=True)
    os.makedirs(os.path.join(validation_dir, category), exist_ok=True)

# Function to split data and move files
def split_data(source_folder, train_folder, validation_folder, validation_split=0.2):
    files = os.listdir(source_folder)
    train_files, validation_files = train_test_split(files, test_size=validation_split)

    # Copy files to training directory
    for file in train_files:
        shutil.copy(os.path.join(source_folder, file), os.path.join(train_folder, file))

    # Copy files to validation directory
    for file in validation_files:
        shutil.copy(os.path.join(source_folder, file), os.path.join(validation_folder, file))

# Apply function to original and distorted datasets
split_data(original_dir, os.path.join(train_dir, 'original'), os.path.join(validation_dir, 'original'))
split_data(rotated_dir, os.path.join(train_dir, 'rotated'), os.path.join(validation_dir, 'rotated'))

In [7]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_directory(
    'dataset/train',
    target_size=(224, 224),
    batch_size=5,
    class_mode='categorical'
)

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
validation_generator = validation_datagen.flow_from_directory(
    'dataset/validation',
    target_size=(224, 224),
    batch_size=5,
    class_mode='categorical'
)

Found 116 images belonging to 2 classes.
Found 30 images belonging to 2 classes.


In [8]:
import shutil

def remove_unwanted_dirs(directory):
    checkpoint_dir = os.path.join(directory, '.ipynb_checkpoints')
    if os.path.exists(checkpoint_dir):
        shutil.rmtree(checkpoint_dir)
        print(f"Removed {checkpoint_dir}")

remove_unwanted_dirs(train_dir)
remove_unwanted_dirs(validation_dir)

print("Updated training directories:", os.listdir(train_dir))
print("Updated validation directories:", os.listdir(validation_dir))

Updated training directories: ['original', 'rotated']
Updated validation directories: ['original', 'rotated']


In [9]:
dataset_dir = 'dataset'
print("Directories in dataset:", os.listdir(dataset_dir))

for category in os.listdir(dataset_dir):
    category_path = os.path.join(dataset_dir, category)
    if os.path.isdir(category_path):
        print(f"Contents of {category}:", os.listdir(category_path))

Directories in dataset: ['original', 'validation', 'train', 'rotated']
Contents of original: ['20.jpg', '13.jpg', '06.jpg', '01.jpg', '03.jpg', '18.jpg', '04.jpg', '15.jpg', '16.jpg', '07.jpg', '11.jpg', '09.jpg', '14.jpg', '02.jpg', '19.jpg', '10.jpg', '12.jpg', '08.jpg', '05.jpg', '17.jpg']
Contents of validation: ['original', 'rotated']
Contents of train: ['original', 'rotated']
Contents of rotated: ['rot_0_2290.jpeg', 'rot_0_100.jpeg', 'rot_0_2498.jpeg', 'rot_0_4857.jpeg', 'rot_0_3397.jpeg', 'rot_0_4570.jpeg', 'rot_0_2198.jpeg', 'rot_0_2929.jpeg', 'rot_0_1574.jpeg', 'rot_0_7418.jpeg', 'rot_0_7875.jpeg', 'rot_0_7912.jpeg', 'rot_0_2258.jpeg', 'rot_0_9038.jpeg', 'rot_0_1084.jpeg', 'rot_0_1167.jpeg', 'rot_0_2785.jpeg', 'rot_0_3981.jpeg', 'rot_0_9267.jpeg', 'rot_0_6281.jpeg', 'rot_0_6478.jpeg', 'rot_0_6029.jpeg', 'rot_0_7464.jpeg', 'rot_0_153.jpeg', 'rot_0_1658.jpeg', 'rot_0_65.jpeg', 'rot_0_806.jpeg', 'rot_0_8217.jpeg', 'rot_0_7093.jpeg', 'rot_0_6015.jpeg', 'rot_0_1981.jpeg', 'rot_0_97

In [10]:
import shutil
import os

directories_to_remove = ['original', 'rotated']
base_dir = 'dataset'

for dir_name in directories_to_remove:
    dir_path = os.path.join(base_dir, dir_name)
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
        print(f"Removed directory: {dir_path}")
    else:
        print(f"Directory does not exist, no need to remove: {dir_path}")

print("Updated contents of the dataset directory:", os.listdir(base_dir))

Removed directory: dataset/original
Removed directory: dataset/rotated
Updated contents of the dataset directory: ['validation', 'train']


In [11]:
!pip install tensorflow_addons
import tensorflow_addons as tfa

# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
x = Flatten()(base_model.output)
x = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(x)  # L2
x = Dropout(0.5)(x)  # Dropout
predictions = Dense(1, activation='sigmoid')(x)

optimizer = tfa.optimizers.AdamW(learning_rate=0.0001, weight_decay=0.001)

# Compile the model
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Prepare training data
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2  # Use 20% of the data as validation set
)

train_generator = train_datagen.flow_from_directory(
    'dataset',
    target_size=(224, 224),
    batch_size=5,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'dataset',
    target_size=(224, 224),
    batch_size=5,
    class_mode='binary',
    subset='validation'
)

# Use early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', restore_best_weights=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 4.1 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


58889256/58889256 [==============================] - 0s 0us/step
Found 117 images belonging to 2 classes.
Found 29 images belonging to 2 classes.


In [12]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=15,
    epochs=10,
    callbacks=[early_stopping],
    validation_data=validation_generator,
    validation_steps=5
)

# Evaluate the model
results = model.evaluate(validation_generator)
print("Test loss, Test accuracy:", results)

Epoch 1/10
15/15 [==============================] - 79s 5s/step - loss: 6.9632 - accuracy: 0.7500 - val_loss: 5.7785 - val_accuracy: 0.6400
Epoch 2/10
15/15 [==============================] - 67s 5s/step - loss: 4.0698 - accuracy: 0.8000 - val_loss: 7.6242 - val_accuracy: 0.6000
Epoch 3/10
15/15 [==============================] - 62s 4s/step - loss: 6.4777 - accuracy: 0.8333 - val_loss: 8.8830 - val_accuracy: 0.6400
Epoch 4/10
15/15 [==============================] - 66s 4s/step - loss: 3.9750 - accuracy: 0.8056 - val_loss: 8.9330 - val_accuracy: 0.7200
Epoch 5/10
15/15 [==============================] - 70s 5s/step - loss: 3.2138 - accuracy: 0.8750 - val_loss: 7.2689 - val_accuracy: 0.7200
Epoch 6/10
15/15 [==============================] - 65s 4s/step - loss: 1.5172 - accuracy: 0.9200 - val_loss: 8.8473 - val_accuracy: 0.6800
Epoch 7/10
15/15 [==============================] - 66s 4s/step - loss: 1.3746 - accuracy: 0.9306 - val_loss: 8.0360 - val_accuracy: 0.7200
Epoch 8/10
15/15 [==